## LAB 1 Jak działa GPT
### Wprowadzenie do biblioteki PyTorch i algorytmów klasyfikacji

Celem laboratorium jest:
* implementacja prostego liniowego algorytmu klasyfikacji z wykorzystaniem biblioteki PyTorch
* implementacja algorytmu optymalizacji gradientowej
* zapozanienie z podstawami biblioteki PyToch

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

### Dane uczące
* generujemy dane (X) - 300 przykładów o dwóch cechach
* oraz prawidłowe etykiety y - dla trzech klas

In [ ]:
def generate_data(n_samples=300):
    X = torch.rand(n_samples, 2) * 10 - 5 
    
    y = torch.zeros(n_samples, dtype=torch.long)
    y[X[:, 1] > 2 * X[:, 0] + 4] = 0  # Klasa 0
    y[X[:, 1] < 2 * X[:, 0] - 4] = 1  # Klasa 1
    y[(X[:, 1] <= 2 * X[:, 0] + 4) & (X[:, 1] >= 2 * X[:, 0] - 4)] = 2  # Klasa 2
    
    return X, y

In [ ]:
X, y = generate_data()

In [ ]:
# kształt X
X.shape

In [ ]:
# 5 pierwszych przykładów
X[:5]

In [ ]:
# 5 pierwszych etykiet
y[:5]

##### Klasyfikacja
Zadaniem algorytmu jest klasyfikacja kropek do trzech kategorii (0 - żółty, 1 - zielony, 2 - czerwony)

In [ ]:
colors = ['#FFCC00', '#66CC99', '#FF6666']
cmap = ListedColormap(colors[:len(torch.unique(y))])
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap);
classes = np.unique(y)
for cls in classes:
    plt.scatter([], [], color=cmap(cls), label=f'Klasa {cls}')

plt.legend(title="Klasy");

Zamiana $y$ na kodowanie one hot

In [ ]:
y_one_hot = F.one_hot(y, num_classes=3)

In [ ]:
# 5 pierwszych zakodowanych etykiet
y_one_hot[:5]

##### Zaimplementuj funkcję softmax
$$softmax(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}$$
* funkcja na wejściu przyjmuje logity
* i zwraca prawdopodobieństwa

In [ ]:
def softmax(logits):
    # TODO 
    return probs

In [ ]:
torch.manual_seed(0);

In [ ]:
logits = torch.normal(mean=0, std=1, size=(2, 3))
logits

In [ ]:
probs = softmax(logits)
probs

In [ ]:
assert torch.allclose(probs, torch.tensor([[0.8446, 0.1349, 0.0205], [0.7511, 0.1438, 0.1051]]), atol = 1e-4)

##### Zaimplementuj entropię krzyżową
$$ loss = -\sum_{i=1}^{n} y_i \log(\hat{y}_i)$$ 
* Funkcja przyjmuje jako argumenty wyliczone prawdopodobieństwa i prawidłowe etykiety kodowane one hot
* Zwraca wartość entropii krzyżowej uśrednionej dla wszystkich przykładów

In [ ]:
def cross_entropy(probs, y_one_hot):
    n_examples = probs.shape[0] # liczba przykładów
    # TODO 
    return loss 

In [ ]:
loss = cross_entropy(probs, torch.tensor([[1, 0, 0], [0, 1, 0]]))
loss

In [ ]:
assert torch.allclose(loss, torch.tensor(1.0540))

#### Pętla uczenia

Dla każdej epoki:
1. Wyznacz logity $$logits = XW + b $$
2. Zastosuj funkcję aktywacji softmax $$probs = softmax(logits)$$
3. Wyznacz wartość funkcji straty $loss$ jako entropię krzyżową
4. Wyznacz gradienty
5. Zaktualizuj wartości wag
   $$ W = W - \alpha \frac{\partial loss}{\partial W}$$
   $$ b = b - \alpha \frac{\partial loss}{\partial b}$$

In [ ]:
alpha = 0.01 # współczynnik uczenia
epochs = 2000 # liczba epok
# inicjalizacja wag
W = torch.randn(2, 3, requires_grad=True)  # Wagi (3 klasy, 2 cechy)
b = torch.randn(3, requires_grad=True)  # Bias

In [ ]:
for epoch in range(epochs):
    logits =  # TODO (1)
    probs =   # TODO (2)
    loss =  # TODO (3)
        
    # TODO (4)
        
    with torch.no_grad():  
        # TODO (5)
        
        W.grad.zero_() # zerujemy gradienty przed kolejną pętlą
        b.grad.zero_()
        
    if epoch % 100 == 0:
        print(f"Epoka {epoch}: Loss = {loss.item():.4f}")

In [ ]:
def plot_decision_boundary(W, b, X, y):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = torch.meshgrid(torch.linspace(x_min, x_max, 100),
                            torch.linspace(y_min, y_max, 100))
    grid = torch.stack([xx.flatten(), yy.flatten()], dim=1)
    logits = grid @ W + b
    preds = torch.argmax(logits, dim=1).reshape(xx.shape)
    
    plt.contourf(xx, yy, preds, alpha=0.3, cmap=cmap)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)
    plt.title("Granice decyzyjne")
    plt.show()

##### Zwizualizujemy granice decyzyjne klasyfikatora
* kropki - nasze dane uczące
* kolor tła - to co decyduje klasyfikator
* jeśli jedno mniej więcej zgadza się z drugim to znaczy, że uczenie przebiegło prawidłowo

In [ ]:
plot_decision_boundary(W, b, X, y);

### Wnioski
Proszę opisać jedną rzecz, która jest niejasna (z wykładu, z labu) (funkcje rysujące wykresy pomijamy)